# Odyssey Tutorial

Welcome to Odyssey! Odyssey is a Python package that can analyze python library usage on GitHub through Google BigQuery. The purpose of this tutorial is to provide you a high-level idea of how to use it. Let's begin!

## Part 1: Work with GithubPython object

We start by introducing a central piece of Odyssey -- GithubPython object. This is the object that connects to Github data using BigQuery. It takes care of all the BigQuery connection, SQL query building, result polling, etc. for you.

Let's start by creating a default GithubPython object. Because we didn't specify any package, the information we will get is about all data in the BigQuery Github database.

In [1]:
from odyssey.core.bigquery.GithubPython import GithubPython
gp = GithubPython()

Let's try to see how many Python files in our BigQuery Github database.

You may think: "Wow that's way less than I expect. Does it mean that we only have ~5.9 million Python files on Github? The answer is no. The main reason is that Google BigQuery only has access to open-sourced repos on Github (those who has certain licences). Therefore, it is just a small subset of the whole Github.

That's why, if you search for *.py file using Github web GUI, the number you will get won't be comparable to the number you get here.

In [2]:
print(gp.get_count())

5995653


Now let's create another GithubPython object, but this time, specify that the package we are interested in is sklearn.

Also, Odyssey allows you to exclude forks of the package, by explicitly providing a list of keywords that shouldn't appear in the repo name or file path. In this case, scikit-learn is the one we should avoid counting.

In [3]:
gp_sklearn = GithubPython(package="sklearn", exclude_forks=["scikit-learn"])

Let's count then how many files that count "sklearn". **Caveat: Note that this is a simple string matching. So even if sklearn appears in comment or as a variable name, it will still count!**

In [4]:
print(gp_sklearn.get_count())

37262


If you want to see exactly what are those 37262 files that contain the word "sklearn", you can use get_all() to see all the entries. The return result is a list of BigQueryGithubEntry, a wrapper that provides nice utility function, such as get_url().

In [5]:
data = gp_sklearn.get_all()

In [6]:
print(type(data[0]))

<class 'odyssey.core.bigquery.BigQueryGithubEntry.BigQueryGithubEntry'>


In [7]:
from odyssey.utils.output import pprint_ipynb

In [8]:
pprint_ipynb(data[0])

In [9]:
data[0].get_url() # a link to Github file

'https://github.com/MikkelsCykel/CTFBD/tree/master/week_04/ex41.py'

## Part 2: Use filter to refine search

Sometimes we are interested in searching for code snippet that contains usage of a specific class. In other cases, the criteria is a little bit more complicated, such as having "X" function and "Y" function in one file, or having "Z" alone. To support those need, filter is built. Let's now utilize its power to refine the search!

In [10]:
from odyssey.core.bigquery.filter import Contains, And, Or

In [11]:
# Let's define a filter that asks for either RandomForestClassifier or RandomForestRegressor
rf_classifier_or_regressor = Or(Contains('RandomForestRegressor'),Contains('RandomForestClassifier'))

In [12]:
# Then another filter that asks for occurence of SVC
svc = Contains('SVC')

In [13]:
# Connect the two using And
# so we are interested in files that have both SVC and one of the two RandomForest models 
# (RandomForestClassifier or RandomForestRegressor) appearing at the same time.
f = And(rf_classifier_or_regressor, svc)

In [14]:
rf_and_svc = gp_sklearn.get_all(f)

In [15]:
print(len(rf_and_svc))

1070


In [16]:
# Verify the occurence. We indeed have both!
pprint_ipynb(rf_and_svc[0])

## Part 3: Repos with top imports

One common question Python library writers (or even users) are interested in is: who is using this library? Odyssey supports querying repos with top imports of your package-in-interest. In one line, you can get the answer!

*Sidenote: Some profiling on parso.parse. It's actually pretty slow!*

In [17]:
import cProfile
import parso
cProfile.run('parso.parse(data[0].code)')

         112786 function calls (109102 primitive calls) in 0.285 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
     1763    0.001    0.000    0.003    0.000 <string>:12(__new__)
        1    0.000    0.000    0.285    0.285 __init__.py:49(parse)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:259(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:308(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:318(decode)
        1    0.000    0.000    0.000    0.000 grammar.py:154(_get_token_namespace)
        1    0.000    0.000    0.215    0.215 grammar.py:204(__init__)
        1    0.000    0.000    0.000    0.000 grammar.py:213(_tokenize_lines)
        1    0.000    0.000    0.217    0.217 grammar.py:249(load_grammar)
        1    0.000   

In [18]:
gp_sklearn_limited = GithubPython(package="sklearn", exclude_forks=["scikit-learn"], limit=100)

In [19]:
top20_imports = gp_sklearn_limited.get_top_import_repo() # top imports by file count

0


In [20]:
print(top20_imports)

[('dallascard/guac', 3), ('automl/paramsklearn', 2), ('ngoix/OCRF', 2), ('cocoaaa/ml_gesture', 2), ('wjfwzzc/Kaggle_Script', 2), ('JoostVisser/ml-assignment2', 1), ('fukatani/PreTrainingChain', 1), ('Mustyy/Udacity-DeepLearning', 1), ('rmonat/princess-or-frog', 1), ('reidbradley/prospecting', 1), ('gpiatkovska/Machine-Learning-in-Python', 1), ('neogi/machine-learning', 1), ('dotsdl/msmbuilder', 1), ('all-umass/metric-learn', 1), ('jmorton/yatsm', 1), ('100star/h2o', 1), ('CharLLCH/Word2Vec', 1), ('haradatm/nlp', 1), ('GbalsaC/bitnamiP', 1), ('MIREL-UNC/wikipedia-ner', 1), ('NicovincX2/Python-3.5', 1), ('antoinecarme/pyaf', 1), ('MartinThoma/algorithms', 1), ('jashwanth9/Expert-recommendation-system', 1), ('rcln/tag.suggestion', 1), ('adazey/Muzez', 1), ('dav-stott/phd-thesis', 1), ('afruizc/microsoft_malware_challenge', 1), ('Edouard360/text-mining-challenge', 1), ('LaRiffle/axa_challenge', 1), ('antlr/codebuff', 1), ('vibhutiM/Production-Failures', 1), ('AppleFairy/machinelearning', 1

In [21]:
# Verify that the the count matches
print(sum([count for _, count in top20_imports])) # 100

100


## Part 4: Most imported class/submodule/funcion

Another common question is how often a certain class/submodule/function is imported. Odyssey can answer that too.

In [22]:
top20_models = gp_sklearn_limited.get_most_imported_class(n=20)

In [23]:
print(top20_models)

[('RandomForestClassifier', 11), ('LogisticRegression', 7), ('CountVectorizer', 6), ('SVC', 6), ('PCA', 5), ('TfidfTransformer', 5), ('LabelEncoder', 4), ('GridSearchCV', 4), ('GaussianNB', 3), ('TfidfVectorizer', 3), ('StandardScaler', 3), ('MultinomialNB', 3), ('Pipeline', 3), ('RandomForestRegressor', 3), ('MinMaxScaler', 2), ('GradientBoostingClassifier', 2), ('KNeighborsClassifier', 2), ('Imputer', 2), ('GradientBoostingRegressor', 2), ('Ridge', 2)]


See what are the entries by calling get_import_source() 

In [24]:
sources = gp_sklearn_limited.get_import_source("RandomForestClassifier")

In [25]:
pprint_ipynb(sources[0])

## Part 5: Instantiation

For classes, Odyssey can provide you with insights about how they are instantiated, default argument value people use, etc.

**Note: All the arguments in the returned dictionary are in string format (even for integer values). This may be changed later.**

In [26]:
rfc_instantiation = gp_sklearn_limited.get_instantiation("RandomForestClassifier")

In [27]:
rfc_instantiation

defaultdict(<function odyssey.core.analyzer.InstantiationAnalyzer.InstantiationAnalyzer.__init__.<locals>.<lambda>>,
            {'**': defaultdict(int, {None: 1}),
             '**classif_base.get_params()': defaultdict(int, {None: 1}),
             '**cls_kwargs': defaultdict(int, {None: 1}),
             '**params': defaultdict(int, {None: 1}),
             '**rf_config': defaultdict(int, {None: 1}),
             '**rf_parameters': defaultdict(int, {None: 1}),
             'bootstrap': defaultdict(int,
                         {'False': 1,
                          'True': 1,
                          'estimator.best_estimator_.bootstrap': 1}),
             'class_weight': defaultdict(int,
                         {'"balanced"': 2, "'balanced'": 1, 'None': 1}),
             'compute_importances': defaultdict(int, {'True': 1}),
             'criterion': defaultdict(int,
                         {'"gini"': 1,
                          "'entropy'": 2,
                          "'gini'"